In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

In [8]:

def mergeDatasets(path):	#takes the path and returns a merged dataset
    data_m = pd.read_csv(path+'movies.csv')
    data_r = pd.read_csv(path+'ratings.csv')
    data_u = pd.read_csv(path+'users.csv')

    movie_ratings = pd.merge(data_m, data_r)
    lens = pd.merge(movie_ratings, data_u)
    #print(lens.head())

    #most_rated = lens.groupby('title').size().sort_values(ascending=False)[:25]
    #print(most_rated)

    return lens

def createUserItemPair(data):
    ## picks top k rated movies & top k users with most ratings
    ## selects subset containing only above users and movies
    ## creates (user, item) pair matrix with rating as cell value
    lens = data.copy()

    most_rated_users = lens.groupby('userId').size().sort_values(ascending=False)
    mru_keys = list(most_rated_users.keys())

    most_rated_movies = lens.groupby('itemId').size().sort_values(ascending=False)
    mrm_keys = list(most_rated_movies.keys())


    data = data.loc[data['userId'].isin(mru_keys) & data['itemId'].isin(mrm_keys)]
    data = data.pivot(index='userId', columns='itemId', values='rating')
    data = np.where(np.isnan(data),0, data)
    #print(data.head())
    return data

def createMatrixI(data):
    I = data.copy()
    I = np.asarray(I)
    I = np.where(np.isnan(I), 0, 1)
    
    return I    


In [11]:
def user_user_similarity_matrix(data):
    uu_matrix = cosine_similarity(data)
    return uu_matrix
    
def prediction_matrix(A, uu_matrix):
    pred_matrix = np.zeros(A.shape)
    K = A.shape[0] #no. of users
    w = A.shape[1] #no. of items
    
    mean_ratings_of_users = []
    for i in range(K):
        mean = np.sum(A[i])/np.count_nonzero(A[i])
        mean_ratings_of_users.append(mean)
    
    for i in range(0, w): #items
        for a in range(0, K): #users
            sum = 0
            for u in range(K):
                sum += (A[u][i] - mean_ratings_of_users[u]) * uu_matrix[a][u]
            sum /= np.sum(uu_matrix[a])
            
            pred_matrix[a][i] = mean_ratings_of_users[a] + sum
            
    return pred_matrix
        
    
def collab_filtering():
    
    pass

In [12]:
def main():
    path = 'data/'
    data = mergeDatasets(path)

    A = createUserItemPair(data)
    I = createMatrixI(A)
    uu_matrix = user_user_similarity_matrix(A)
    pred_matrix = prediction_matrix(A, uu_matrix)

if __name__ == '__main__':
    main()

KeyboardInterrupt: 

In [ ]:
10.16.40